In [5]:
import ray
ray.shutdown()
ray.init(address="ray://localhost:10001")

ClientContext(dashboard_url='127.0.0.1:8265', python_version='3.7.10', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', protocol_version='2022-03-16', _num_clients=3, _context_to_restore=<ray.util.client._ClientContext object at 0x7fac51dfffd0>)

In [6]:
# Before cluster resize
ray.cluster_resources()

{'memory': 27319277570.0,
 'node:172.31.26.25': 1.0,
 'object_store_memory': 12200569650.0,
 'CPU': 12.0,
 'node:172.31.25.104': 1.0,
 'node:172.31.31.134': 1.0}

In [7]:
%%sh

# I was hoping to do a manual resize here, but ala
REGION=$(curl --silent http://169.254.169.254/latest/dynamic/instance-identity/document | jq -r .region)
INSTANCE=`wget -q -O - http://169.254.169.254/latest/meta-data/instance-id`
CLUSTER_ID=$(aws --region ${REGION} ec2 describe-instances --instance-ids $INSTANCE --query 'Reservations[0].Instances[0].Tags[?Key==`aws:elasticmapreduce:job-flow-id`].Value' | jq -r '.[0]')
CORE_INSTANCE_GROUP=$(aws --region ${REGION} emr list-instances --cluster-id ${CLUSTER_ID} --instance-group-types "CORE" | jq -r '.Instances[0].InstanceGroupId')

#echo "Resizing ${CORE_INSTANCE_GROUP} on ${CLUSTER_ID}"
#aws --region ${REGION} emr modify-instance-groups --instance-groups InstanceGroupId=${CORE_INSTANCE_GROUP},InstanceCount=2


Resizing ig-339GIILJH877U on j-THYVRP8QUWDE



An error occurred (AccessDeniedException) when calling the ModifyInstanceGroups operation: User: arn:aws:sts::079002598131:assumed-role/EMR_EC2_DefaultRole/i-04d536328369140a0 is not authorized to perform: elasticmapreduce:ModifyInstanceGroups on resource: * because no identity-based policy allows the elasticmapreduce:ModifyInstanceGroups action


CalledProcessError: Command 'b'\n# I was hoping to do a manual resize here, but ala\nREGION=$(curl --silent http://169.254.169.254/latest/dynamic/instance-identity/document | jq -r .region)\nINSTANCE=`wget -q -O - http://169.254.169.254/latest/meta-data/instance-id`\nCLUSTER_ID=$(aws --region ${REGION} ec2 describe-instances --instance-ids $INSTANCE --query \'Reservations[0].Instances[0].Tags[?Key==`aws:elasticmapreduce:job-flow-id`].Value\' | jq -r \'.[0]\')\nCORE_INSTANCE_GROUP=$(aws --region ${REGION} emr list-instances --cluster-id ${CLUSTER_ID} --instance-group-types "CORE" | jq -r \'.Instances[0].InstanceGroupId\')\n\necho "Resizing ${CORE_INSTANCE_GROUP} on ${CLUSTER_ID}"\n\naws --region ${REGION} emr modify-instance-groups --instance-groups InstanceGroupId=${CORE_INSTANCE_GROUP},InstanceCount=2\n'' returned non-zero exit status 255.

In [ ]:
# After cluster resize
ray.cluster_resources()

In [ ]:
ray.shutdown()

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import ray
ray.shutdown()

import joblib
from ray.util.joblib import register_ray

register_ray()
#ray.init(address="localhost:6379")
ray.init("ray://localhost:10001")

#import mlflow
#mlflow.set_tracking_uri('http://127.0.0.1:5001')
# mlflow.autolog() #log_models=False, exclusive=True)
import sklearn
#mlflow.sklearn.autolog()#log_models=True)

digits = load_digits()

param_space = {
    'C': np.logspace(-6, 6, 30),
    'gamma': np.logspace(-8, 8, 30),
    'tol': np.logspace(-4, -1, 30),
    'class_weight': [None, 'balanced'],
}
model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=2, n_iter=5, verbose=10)

with joblib.parallel_backend('ray'):
    search.fit(digits.data, digits.target)